In [5]:
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')
org_items = len(items)

In [6]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')

ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [7]:
for i in items['movie id']:
    if(len(ratings[ratings['movie_id'] == i]) <= 10):
        items.drop(i-1,inplace = True)
        ratings.drop(ratings[ratings['movie_id'] == i].index,inplace = True)

def predict(ratings, similarity, type='user'):
    
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1).reshape(-1,1)
        
        ratings_diff = (ratings - mean_user_rating)
        pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

In [8]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
print(n_users,n_items)

943 1119


In [9]:
items = items.reset_index(drop = True)

In [10]:
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, items[items['movie id'] == line[2]].index.values[0] ] = line[3]
#         data_matrix[line[1]-1, line[2]-2 ] = line[3]



In [11]:
data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [14]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')


In [15]:
user_prediction = predict(data_matrix, user_similarity, type='user')
item_prediction = predict(data_matrix, item_similarity, type='item')


In [16]:
user_pred = pd.DataFrame(user_prediction)
item_pred = pd.DataFrame(item_prediction)

In [17]:
user_pred

,0,1,2,3,4,5,6,7,8,9,...,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118
0,2.235147,0.905309,0.802637,1.179952,0.813373,0.649508,1.953717,1.332476,1.684132,0.876699,...,0.592456,0.600405,0.605281,0.603287,0.618797,0.600252,0.604867,0.587363,0.605689,0.595257
1,1.721155,0.341706,0.153312,0.689435,0.183163,-0.039437,1.451273,0.834104,1.065142,0.218675,...,-0.095335,-0.083947,-0.078534,-0.081460,-0.066320,-0.088495,-0.079027,-0.103359,-0.090631,-0.093384
2,1.733892,0.265929,0.095588,0.621250,0.110092,-0.099101,1.425780,0.772978,1.072625,0.173057,...,-0.162878,-0.153909,-0.148191,-0.149763,-0.135525,-0.158925,-0.148371,-0.173274,-0.161847,-0.162680
3,1.646599,0.208889,0.042018,0.560581,0.056682,-0.147971,1.352342,0.712612,1.012281,0.126791,...,-0.211812,-0.201886,-0.198297,-0.199491,-0.184737,-0.208753,-0.197433,-0.222600,-0.209046,-0.211247
4,1.833881,0.493106,0.394702,0.800791,0.399657,0.236958,1.585258,0.946607,1.331683,0.478224,...,0.170942,0.178991,0.178287,0.178943,0.195748,0.172551,0.183013,0.160858,0.179643,0.172466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1.634921,0.304397,0.135463,0.648047,0.157835,-0.038829,1.387420,0.789011,1.028555,0.220079,...,-0.100817,-0.090443,-0.086544,-0.089210,-0.071395,-0.098126,-0.086420,-0.110210,-0.093377,-0.100714
939,1.834629,0.430399,0.298027,0.726663,0.305992,0.117922,1.526185,0.863983,1.206493,0.370406,...,0.054573,0.065137,0.069030,0.068257,0.080032,0.062828,0.069129,0.048257,0.062057,0.056653
940,1.512465,0.196274,0.023087,0.545502,0.054860,-0.148657,1.241485,0.686661,0.955857,0.113594,...,-0.210165,-0.201302,-0.196955,-0.198497,-0.182763,-0.207030,-0.195755,-0.221167,-0.204883,-0.209526
941,1.811383,0.405065,0.276039,0.726800,0.281799,0.087434,1.551089,0.850030,1.206129,0.343305,...,0.021760,0.033368,0.033057,0.036058,0.042393,0.028189,0.036876,0.014899,0.028845,0.023801


In [19]:
item_pred

,0,1,2,3,4,5,6,7,8,9,...,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118
0,0.695108,0.732261,0.768520,0.691876,0.776530,0.804270,0.693761,0.715756,0.729759,0.776204,...,0.827081,0.822338,0.826941,0.826365,0.830749,0.847956,0.821433,0.843702,0.840898,0.832929
1,0.173203,0.209972,0.194437,0.200298,0.202680,0.192151,0.176658,0.193793,0.171772,0.186274,...,0.203740,0.205072,0.208352,0.205157,0.205723,0.210255,0.206262,0.211054,0.197508,0.205234
2,0.140326,0.147867,0.138660,0.147642,0.141411,0.136493,0.136492,0.145628,0.136947,0.138308,...,0.142039,0.137547,0.143169,0.141055,0.140575,0.141523,0.139378,0.142590,0.128935,0.139171
3,0.076324,0.084774,0.081256,0.083995,0.082965,0.083616,0.075500,0.083675,0.078705,0.084403,...,0.086742,0.085210,0.086677,0.085318,0.086304,0.085047,0.084627,0.087019,0.080592,0.085349
4,0.349602,0.352207,0.399179,0.352867,0.393322,0.433752,0.362238,0.366299,0.397460,0.413498,...,0.427081,0.422173,0.403575,0.412337,0.426602,0.415487,0.417341,0.413990,0.425756,0.425065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.151608,0.183636,0.174363,0.183868,0.177861,0.186784,0.161154,0.179896,0.158921,0.185671,...,0.190875,0.190198,0.190900,0.185610,0.194984,0.185581,0.188283,0.192501,0.186315,0.185153
939,0.269168,0.298175,0.310883,0.270590,0.308917,0.318010,0.266209,0.268786,0.276119,0.304603,...,0.320448,0.324534,0.323808,0.326377,0.318625,0.335160,0.322685,0.332467,0.321691,0.322650
940,0.052898,0.072610,0.067910,0.070933,0.074907,0.077477,0.053558,0.069217,0.063336,0.074252,...,0.081450,0.078837,0.080494,0.079843,0.081172,0.080677,0.079919,0.081057,0.078125,0.080482
941,0.258395,0.280760,0.299015,0.268342,0.293586,0.294498,0.269593,0.254769,0.268526,0.283870,...,0.291866,0.299990,0.287385,0.298291,0.280405,0.303767,0.297280,0.303241,0.292097,0.293820


In [20]:
max(item_pred.iloc[0])

0.8569895316900752

In [21]:
item_pred.iloc[1][item_pred.iloc[1]==max(item_pred.iloc[1])][:1]

74    0.217
Name: 1, dtype: float64

In [22]:
user_similarity[0].max()

0.9972725491936792

In [23]:
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [18]:
k = items['movie id'][[1,2,4,5,1118]]
[items[items['movie id'] == k.values[i]].index.values[0] for i in range(len(k))]

[1, 2, 4, 5, 1118]

In [19]:
k

1          2
2          3
4          5
5          6
1118    1518
Name: movie id, dtype: int64

In [25]:
first_h = 10
cols = items.columns[6:]

def UserUserRecommendation(userid):
    user_indv = ratings[ratings['user_id'] == userid]['movie_id'].values
    user_pred_indv = items['movie id'][user_pred.iloc[userid-1].sort_values(ascending = False).index.values]
    diff = np.setdiff1d(user_pred_indv, user_indv)
    diff = [items[items['movie id'] == diff[i]].index.values[0] for i in range(len(diff))]
    rec_mov = user_pred.iloc[0,diff].sort_values(ascending = False)[:first_h]
    li = [ list(cols[(items.iloc[rec_mov.index.values,6:].values == 1)[i]].values) for i in range(first_h)]
    li = [','.join(i) for i in li]
    return [i+' - '+j for i,j in zip(list(items.iloc[rec_mov.index.values]['movie title']),li)]



In [27]:
UserUserRecommendation(100)

['Star Wars (1977) - Action,Adventure,Romance,Sci-Fi,War',
 'Fargo (1996) - Crime,Drama,Thriller',
 'Return of the Jedi (1983) - Action,Adventure,Romance,Sci-Fi,War',
 'Godfather, The (1972) - Action,Crime,Drama',
 "Toy Story (1995) - Animation,Children's,Comedy",
 'Raiders of the Lost Ark (1981) - Action,Adventure',
 'Silence of the Lambs, The (1991) - Drama,Thriller',
 'Pulp Fiction (1994) - Crime,Drama',
 'Independence Day (ID4) (1996) - Action,Sci-Fi,War',
 'Jerry Maguire (1996) - Drama,Romance']

In [28]:
item_df = pd.DataFrame(item_similarity)

In [29]:
def ItemItemRecommendation(userid):
    pred = None
    for i in ratings[ratings['user_id'] == userid].sort_values(by = ['unix_timestamp'])['movie_id']:
        ind = items[items['movie id'] == i].index.values[0]
        k = item_df[ind]
        k = k.sort_values(ascending = False)
        k = k[:10]
        if type(pred) != pd.Series:
            pred = k
        else:
            pred.append(k)
            pred = pred.sort_values(ascending = False)[:10]

    rec_mov = pred
    lis = [ list(cols[(items.iloc[rec_mov.index.values,6:].values == 1)[i]].values) for i in range(first_h)]
    lis = [','.join(i) for i in lis]
    return [i+' - '+j for i,j in zip(list(items.iloc[rec_mov.index.values]['movie title']),lis)]

In [30]:
ItemItemRecommendation(100)

['Inspector General, The (1949) - Musical',
 'Mark of Zorro, The (1940) - Adventure',
 'Beverly Hillbillies, The (1993) - Comedy',
 'Carrington (1995) - Drama,Romance',
 'Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964) - Drama,Musical',
 "Free Willy 2: The Adventure Home (1995) - Adventure,Children's,Drama",
 'Red Firecracker, Green Firecracker (1994) - Drama',
 'Audrey Rose (1977) - Horror',
 'Meet Me in St. Louis (1944) - Musical',
 'Love Affair (1994) - Drama,Romance']